# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Loan CSV file generated from WeatherPy Folder
weather_data_to_load = "weather_df.csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()
del dropna_weather_data["Unnamed: 0"]
dropna_weather_data.head(20)

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Nikolskoye,RU,1603251110,59.70,30.79,90,100,30.00,2.24
1,Marietta,US,1603251071,33.95,-84.55,1,88,64.40,4.70
2,Clyde River,CA,1603251333,70.47,-68.59,1,74,28.40,14.99
3,Rikitea,PF,1603250944,-23.12,-134.97,28,76,73.92,9.62
4,Pucallpa,PE,1603251335,-8.38,-74.55,80,91,73.83,0.74
5,Yellowknife,CA,1603251101,62.46,-114.35,20,92,23.00,6.93
6,Ust-Maya,RU,1603251338,60.42,134.53,100,97,24.35,5.06
7,Ushuaia,AR,1603250813,-54.80,-68.30,100,86,39.20,8.05
8,Castro,BR,1603250889,-24.79,-50.01,41,91,62.10,5.55
9,Bluff,NZ,1603250925,-46.60,168.33,40,60,57.00,14.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations using latitude and longitude with humidity
locations = dropna_weather_data[["Latitude", "Longitude"]]
humidity = dropna_weather_data["Humidity (%)"].astype(float)

In [12]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Filter vacation with zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Beroroha,MG,1603251348,-21.67,45.17,0,57,75.54,6.31
1,Aswān,EG,1603251636,24.09,32.91,0,24,76.84,8.66
2,Banamba,ML,1603251638,13.55,-7.45,0,26,77.88,4.59
3,Assiut,EG,1603251736,27.18,31.18,0,41,71.64,5.53
4,Oodweyne,SO,1603251918,9.41,45.06,0,24,73.11,2.13
5,Ţahţā,EG,1603251990,26.77,31.50,0,35,71.85,5.53


In [15]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]
vaca_humidity = indexed_perfect_vacation["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Latitude"]
    lng = indexed_perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"
    
    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")
        
# Dataframe with nearest hotel and note hotel name if found
indexed_perfect_vacation["Hotel Name"] = hotels
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Nearest Hotel,Hotel Name
0,Beroroha,MG,1603251348,-21.67,45.17,0,57,75.54,6.31,Nearest hotel not found,Nearest hotel not found
1,Aswān,EG,1603251636,24.09,32.91,0,24,76.84,8.66,Nuba Nile,Nuba Nile
2,Banamba,ML,1603251638,13.55,-7.45,0,26,77.88,4.59,Kassela de Banamba,Kassela de Banamba
3,Assiut,EG,1603251736,27.18,31.18,0,41,71.64,5.53,Florence Hotel Assiut,Florence Hotel Assiut
4,Oodweyne,SO,1603251918,9.41,45.06,0,24,73.11,2.13,Eng Hiirey House,Eng Hiirey House
5,Ţahţā,EG,1603251990,26.77,31.50,0,35,71.85,5.53,Møhãmęd Tãręk 🏠,Møhãmęd Tãręk 🏠


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

In [20]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig

Figure(layout=FigureLayout(height='420px'))